In [122]:
import pandas as pd

fileURL = './breastcancer.csv'
breast_cancer_db = pd.read_csv(fileURL)
breast_cancer_db = breast_cancer_db.ix[:,1:32]

In [2]:
breast_cancer_db.loc[(breast_cancer_db['diagnosis'] == 'B'),'diagnosis'] = 0
breast_cancer_db.loc[(breast_cancer_db['diagnosis'] == 'M'),'diagnosis'] = 1
breast_cancer_db['diagnosis'] = pd.to_numeric(breast_cancer_db['diagnosis'])

In [3]:
def get_X_y(feature_cols, target):
    X = breast_cancer_db[feature_cols]
    y = breast_cancer_db[target]
    return X, y

In [4]:
feature_cols = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean',
               'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean',
               'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se',
               'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
               'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
               'smoothness_worst', 'smoothness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst',
               'fractal_dimension_worst']

target = 'diagnosis'
X, y = get_X_y(feature_cols, target)

In [5]:
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()

In [6]:
from sklearn.linear_model import LogisticRegression
lrg_clf = LogisticRegression(C=10, fit_intercept=False, tol=0.0001, class_weight='balanced')

In [7]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance')

In [8]:
from sklearn import tree
dt_clf = tree.DecisionTreeClassifier(criterion = "entropy", min_samples_split=3, 
                                     min_samples_leaf=3)

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(criterion = "gini", n_estimators=15)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA 

pca = PCA(n_components=10)

def build_pipeline(clf):
    pipe = Pipeline(steps=[('pca', pca), ('clf', clf)])
    return pipe

In [11]:
from sklearn.cross_validation import cross_val_score

def model_and_validate(clf):
    pipe = build_pipeline(clf)
    scores = cross_val_score(pipe, X, y, cv=10, scoring='accuracy') 
    return scores.mean()

//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
print (model_and_validate(gnb_clf))

0.906972171809


In [15]:
print (model_and_validate(lrg_clf))

0.949083916688


In [16]:
print (model_and_validate(dt_clf))

0.917435830957


In [17]:
print (model_and_validate(knn_clf))

0.931689136635


In [18]:
print (model_and_validate(rf_clf))

0.945513568404


In [20]:
from sklearn.cluster import KMeans
km_cls = KMeans(n_clusters=1)

In [21]:
print (model_and_validate(km_cls))

0.627662907268


In [22]:
from mlxtend.classifier import StackingClassifier

stk_clf = StackingClassifier(classifiers=[rf_clf, lrg_clf], meta_classifier=lrg_clf)

In [21]:
print model_and_validate(stk_clf)

0.947360859044


In [129]:
from sklearn.model_selection import train_test_split
import queue
def model_and_Evaluation(clf):
    pipe = build_pipeline(clf)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,stratify=y)
    pipe.fit(X_train,y_train)
    predict_proba = pipe.predict_proba(X_test)
    q = queue.PriorityQueue()
    for i in range(len(y_test)):
        prob = predict_proba[:,1][i]
        q.put((-prob,y_test.tolist()[i]))
        
    for i in range(20):
        tuple = q.get()
        print(-tuple[0],":", tuple[1])
    

In [130]:
model_and_Evaluation(stk_clf)

0.999059759899 : 0
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.997201044769 : 1
0.5 : 0
0.5 : 0
0.5 : 0


In [131]:
model_and_Evaluation(lrg_clf)

1.0 : 1
1.0 : 1
1.0 : 1
0.999999999999 : 1
0.999999997251 : 1
0.999999519348 : 1
0.999999432333 : 1
0.999998795809 : 1
0.999996266888 : 1
0.999996039685 : 1
0.999984356412 : 1
0.999966777674 : 1
0.999640849543 : 1
0.999531067783 : 1
0.999502034204 : 1
0.996591082143 : 1
0.987455423208 : 1
0.857024479205 : 1
0.825927152771 : 1
0.693481536784 : 0
